# Projeto de Bases de Dados - Parte 3

### Docente Responsável

Prof. Daniel Faria

### Grupo 37
<dl>
    <dt>25 horas (33.3%)</dt>
    <dd>ist1103131 Joana Marques</dd>
    <dt>25 horas (33.3%)</dt>
    <dd>ist1102890 David Nunes</dd>
    <dt>25 horas (33.3%)</dt>
    <dd>ist1103145 Nathaniel Prazeres</dd>
<dl>

In [ ]:
%load_ext sql
%sql postgresql://db:db@postgres/db

# Empresa de comércio online

## 0. Carregamento da Base de Dados

Carregue o esquema de Base de Dados apresentado no Anexo A.

In [ ]:
%%sql

DROP TABLE IF EXISTS customer CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS pay CASCADE;
DROP TABLE IF EXISTS employee CASCADE;
DROP TABLE IF EXISTS process CASCADE;
DROP TABLE IF EXISTS department CASCADE;
DROP TABLE IF EXISTS workplace CASCADE;
DROP TABLE IF EXISTS works CASCADE;
DROP TABLE IF EXISTS office CASCADE;
DROP TABLE IF EXISTS warehouse CASCADE;
DROP TABLE IF EXISTS product CASCADE;
DROP TABLE IF EXISTS contains CASCADE;
DROP TABLE IF EXISTS supplier CASCADE;
DROP TABLE IF EXISTS delivery CASCADE;

CREATE TABLE customer(
cust_no INTEGER PRIMARY KEY,
name VARCHAR(80) NOT NULL,
email VARCHAR(254) UNIQUE NOT NULL,
phone VARCHAR(15),
address VARCHAR(255)
);

CREATE TABLE orders(
order_no INTEGER PRIMARY KEY,
cust_no INTEGER NOT NULL REFERENCES customer,
date DATE NOT NULL
--order_no must exist in contains
);

CREATE TABLE pay(
order_no INTEGER PRIMARY KEY REFERENCES orders,
cust_no INTEGER NOT NULL REFERENCES customer
);

CREATE TABLE employee(
ssn VARCHAR(20) PRIMARY KEY,
TIN VARCHAR(20) UNIQUE NOT NULL,
bdate DATE,
name VARCHAR NOT NULL
--age must be >=18
);

CREATE TABLE process(
ssn VARCHAR(20) REFERENCES employee,
order_no INTEGER REFERENCES orders,
PRIMARY KEY (ssn, order_no)
);

CREATE TABLE department(
name VARCHAR PRIMARY KEY
);

CREATE TABLE workplace(
address VARCHAR PRIMARY KEY,
lat NUMERIC(8, 6) NOT NULL,
long NUMERIC(9, 6) NOT NULL,
UNIQUE(lat, long)
--address must be in warehouse or office but not both
);

CREATE TABLE office(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE warehouse(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE works(
ssn VARCHAR(20) REFERENCES employee,
name VARCHAR(200) REFERENCES department,
address VARCHAR(255) REFERENCES workplace,
PRIMARY KEY (ssn, name, address)
);

CREATE TABLE product(
SKU VARCHAR(25) PRIMARY KEY,
name VARCHAR(200) NOT NULL,
description VARCHAR,
price NUMERIC(10, 2) NOT NULL,
ean NUMERIC(13) UNIQUE
);

CREATE TABLE contains(
order_no INTEGER REFERENCES orders,
SKU VARCHAR(25) REFERENCES product,
qty INTEGER,
PRIMARY KEY (order_no, SKU)
);

CREATE TABLE supplier(
TIN VARCHAR(20) PRIMARY KEY,
name VARCHAR(200),
address VARCHAR(255),
SKU VARCHAR(25) REFERENCES product,
date DATE
);

CREATE TABLE delivery(
address VARCHAR(255) REFERENCES warehouse,
TIN VARCHAR(20) REFERENCES supplier,
PRIMARY KEY (address, TIN)
);

Crie as instruções para o seu preenchimento de forma consistente, garantindo que todas as consultas SQL e OLAP, apresentadas mais adiante, produzam um resultado não vazio. 

In [ ]:
%%sql

INSERT INTO customer (cust_no, name, email, phone, address) VALUES
(1, 'John Doe', 'johndoe@example.com', '123-456-0', '123 Main St'),
(2, 'Jane Smith', 'janesmith@example.com', '555-555-5555', '456 Oak Ave'),
(3, 'Bob Johnson', 'bjohnson@example.com', '555--4567', '789 Elm St');

BEGIN;
INSERT INTO orders (order_no, cust_no, date) VALUES
(1, 1, '2022-06-01'),
(2, 2, '2022-06-02'),
(3, 3, '2020-06-03'),
(4, 1, '2019-06-04'),
(5, 2, '2019-06-05');

INSERT INTO product (SKU, name, description, price, ean) VALUES
('SKU123', 'Product 1', 'This is product 1', 10.99, 1234567890123),
('SKU456', 'Product 2', 'This is product 2', 19.99, 2345678901234),
('SKU789', 'Product 3', 'This is product 3', 5.99, 3456789012345);

INSERT INTO contains (order_no, SKU, qty) VALUES
(1, 'SKU123', 2),
(2, 'SKU456', 1),
(3, 'SKU789', 3),
(4, 'SKU123', 1),
(5, 'SKU456', 2);
COMMIT;

INSERT INTO pay (order_no, cust_no) VALUES
(2, 2),
(3, 3);

INSERT INTO employee (ssn, TIN, bdate, name) VALUES
('123-45-6789', '987-65-4321', '1990-01-01', 'John Smith'),
('234-56-7890', '876-54-3210', '1995-02-02', 'Jane Doe'),
('345-67-8901', '765-43-2109', '2000-03-03', 'Bob Johnson');

INSERT INTO process (ssn, order_no) VALUES
('123-45-6789', 1),
('123-45-6789', 2),
('345-67-8901', 3);

INSERT INTO department (name) VALUES
('Sales'),
('Marketing'),
('Engineering');

BEGIN;
INSERT INTO workplace (address, lat, long) VALUES
('123 Main St', 40.7128, -74.0061),
('456 Oak Ave', 37.7749, -122.4114),
('789 Elm St', 34.0522, -118.2437);

INSERT INTO office (address) VALUES
('123 Main St'),
('456 Oak Ave');

INSERT INTO warehouse (address) VALUES
('789 Elm St');
COMMIT;

INSERT INTO works (ssn, name, address) VALUES
('123-45-6789', 'Sales', '123 Main St'),
('234-56-7890', 'Marketing', '456 Oak Ave'),
('345-67-8901', 'Engineering', '789 Elm St');

INSERT INTO supplier (TIN, name, address, SKU, date) VALUES
('987-65-4321', 'Supplier 1', '123 Main St', 'SKU123', '2023-06-01'),
('876-54-3210', 'Supplier 2', '456 Oak Ave', 'SKU456', '2023-06-02'),
('765-43-2109', 'Supplier 3', '789 Elm St', 'SKU789', '2023-06-03');

INSERT INTO delivery (address, TIN) VALUES
('789 Elm St', '987-65-4321'),
('789 Elm St', '876-54-3210'),
('789 Elm St', '765-43-2109');

BEGIN;
INSERT INTO customer (cust_no, name, email, phone, address) VALUES
(4, 'Sarah Johnson', 'sarah@example.com', '555-123-4567', '789 Elm St'),
(5, 'Michael Smith', 'michael@example.com', '555-987-6543', '456 Oak Ave'),
(6, 'Emma Wilson', 'emma@example.com', '555-111-2222', '789 Main St'),
(7, 'Christopher Davis', 'chris@example.com', '555-333-4444', '456 Elm St');

INSERT INTO orders (order_no, cust_no, date) VALUES
(6, 4, '1022-06-06'),
(7, 5, '1022-06-07'),
(8, 6, '1022-06-08'),
(9, 7, '1022-06-09');

INSERT INTO product (SKU, name, description, price, ean) VALUES
('SKU987', 'Product 4', 'This is product 4', 7.99, 4567890123456),
('SKU653', 'Product 5', 'This is product 5', 12.99, 5678901234567),
('SKU321', 'Product 6', 'This is product 6', 9.99, 6789012345678),
('SKU654', 'Product 7', 'This is product 7', 15.99, 7890123456789);

INSERT INTO contains (order_no, SKU, qty) VALUES
(6, 'SKU987', 2),
(7, 'SKU653', 1),
(8, 'SKU321', 3),
(9, 'SKU654', 2);

INSERT INTO pay (order_no, cust_no) VALUES
(4, 1),
(5, 2),
(8, 6),
(9, 7);

INSERT INTO employee (ssn, TIN, bdate, name) VALUES
('456-78-9012', '654-32-1098', '1998-04-04', 'Emily Davis'),
('567-89-0123', '543-21-0987', '1992-05-05', 'Alex Johnson'),
('678-90-1234', '432-10-9876', '1996-06-06', 'Olivia Anderson'),
('789-01-2345', '321-09-8765', '1994-07-07', 'William Thompson');

INSERT INTO process (ssn, order_no) VALUES
('456-78-9012', 4),
('567-89-0123', 5),
('123-45-6789', 6),
('123-45-6789', 7),
('678-90-1234', 8),
('789-01-2345', 9);

INSERT INTO department (name) VALUES
('Human Resources'),
('Finance'),
('Operations'),
('IT');

INSERT INTO workplace (address, lat, long) VALUES
('101 Elm St', 41.8781, -87.6298),
('202 Oak Ave', 38.9072, -77.0372),
('303 Main St', 37.7749, -122.4124),
('404 Pine St', 34.0522, -118.2432);

INSERT INTO office (address) VALUES
('101 Elm St'),
('202 Oak Ave'),
('303 Main St');

INSERT INTO warehouse (address) VALUES
('404 Pine St');

INSERT INTO works (ssn, name, address) VALUES
('456-78-9012', 'Human Resources', '101 Elm St'),
('567-89-0123', 'Finance', '202 Oak Ave'),
('678-90-1234', 'Operations', '303 Main St'),
('789-01-2345', 'IT', '404 Pine St');

INSERT INTO supplier (TIN, name, address, SKU, date) VALUES
('654-32-1098', 'Supplier 4', '101 Elm St', 'SKU987', '2023-06-04'),
('543-21-0987', 'Supplier 5', '202 Oak Ave', 'SKU653', '2023-06-05'),
('432-10-9876', 'Supplier 6', '303 Main St', 'SKU321', '2023-06-06'),
('321-09-8765', 'Supplier 7', '404 Pine St', 'SKU654', '2023-06-07');

INSERT INTO delivery (address, TIN) VALUES
('404 Pine St', '654-32-1098'),
('404 Pine St', '543-21-0987'),
('404 Pine St', '432-10-9876'),
('404 Pine St', '321-09-8765');
COMMIT;

## 1. Restrições de Integridade

Apresente o código para implementar as seguintes restrições de integridade, se necessário, com recurso a extensões procedimentais SQL (Stored Procedures e Triggers):

(RI-1) Nenhum empregado pode ter menos de 18 anos de idade

In [ ]:
%%sql
-- (RI-1)
ALTER TABLE employee
ADD CONSTRAINT RI_1
CHECK (DATE_PART('year', age(bdate)) >= 18);


(RI-2) Um 'Workplace' é obrigatoriamente um 'Office' ou 'Warehouse' mas não pode ser ambos

In [ ]:
%%sql
-- (RI-2)
CREATE OR REPLACE FUNCTION check_workplace_type() RETURNS TRIGGER AS $$
BEGIN
    IF TG_TABLE_NAME = 'office' THEN
        IF EXISTS (SELECT 1 FROM warehouse WHERE address = NEW.address) THEN
            RAISE EXCEPTION 'A workplace cannot be both an office and a warehouse';
        END IF;
    ELSIF TG_TABLE_NAME = 'warehouse' THEN
        IF EXISTS (SELECT 1 FROM office WHERE address = NEW.address) THEN
            RAISE EXCEPTION 'A workplace cannot be both an office and a warehouse';
        END IF;
    ELSIF NOT EXISTS (SELECT 1 FROM office WHERE address = NEW.address) AND NOT EXISTS (SELECT 1 FROM warehouse WHERE address = NEW.address) THEN
        RAISE EXCEPTION 'A workplace must be either an office or a warehouse';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER check_office_workplace
    BEFORE INSERT OR UPDATE ON office
    FOR EACH ROW
    EXECUTE FUNCTION check_workplace_type();

CREATE TRIGGER check_warehouse_workplace
    BEFORE INSERT OR UPDATE ON warehouse
    FOR EACH ROW
    EXECUTE FUNCTION check_workplace_type();

CREATE CONSTRAINT TRIGGER check_workplace_insert
    AFTER INSERT OR UPDATE ON workplace
    DEFERRABLE INITIALLY DEFERRED
    FOR EACH ROW
    EXECUTE FUNCTION check_workplace_type();


(RI-3) Uma 'Order' tem de figurar obrigatoriamente em 'Contains'.

In [ ]:
%%sql
-- (RI-3)
CREATE OR REPLACE FUNCTION check_contains_order_no() RETURNS TRIGGER AS $$
BEGIN
    IF NOT EXISTS (SELECT 1 FROM contains WHERE order_no = NEW.order_no) THEN
        RAISE EXCEPTION 'Order number % does not exist in the contains table', NEW.order_no;
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE CONSTRAINT TRIGGER check_contains_order_no_trigger
    AFTER INSERT OR UPDATE ON orders
    DEFERRABLE INITIALLY DEFERRED
    FOR EACH ROW
    EXECUTE FUNCTION check_contains_order_no();


## 2. Consultas SQL

Apresente a consulta SQL mais sucinta para cada uma das seguintes questões

1) Qual o número e nome do(s) cliente(s) com maior valor total de encomendas pagas?  

In [ ]:
%%sql

SELECT
    c.cust_no,
    c.name
FROM
    customer c
JOIN
    orders o ON c.cust_no = o.cust_no
JOIN
    pay p ON o.order_no = p.order_no
JOIN
    contains cn ON o.order_no = cn.order_no
JOIN
    product pr ON cn.SKU = pr.SKU
GROUP BY
    c.cust_no,
    c.name
HAVING
    SUM(pr.price * cn.qty) = (
        SELECT
            MAX(total_order_value)
        FROM
            (SELECT
                c.cust_no,
                SUM(pr.price * cn.qty) AS total_order_value
            FROM
                customer c
            JOIN
                orders o ON c.cust_no = o.cust_no
            JOIN
                pay p ON o.order_no = p.order_no
            JOIN
                contains cn ON o.order_no = cn.order_no
            JOIN
                product pr ON cn.SKU = pr.SKU
            GROUP BY
                c.cust_no) AS totals
    );


2. Qual o nome dos empregados que processaram encomendas em todos os dias de 2022 em que houve encomendas?

In [ ]:
%%sql

SELECT e.name
FROM employee e
INNER JOIN process p ON e.ssn = p.ssn
INNER JOIN orders o ON p.order_no = o.order_no
WHERE date_part('year', o.date) = 2022
GROUP BY e.name
HAVING COUNT(DISTINCT date_trunc('day', o.date)) = (
    SELECT COUNT(DISTINCT date_trunc('day', o.date))
    FROM orders o
    WHERE date_part('year', o.date) = 2022
);


3. Quantas encomendas foram realizadas mas não pagas em cada mês de 2022?

In [ ]:
%%sql

SELECT EXTRACT(month FROM o.date) AS month, COUNT(*) AS num_unpaid_orders
FROM orders o
LEFT JOIN pay p ON o.order_no = p.order_no
WHERE EXTRACT(year FROM o.date) = 2022 AND p.order_no IS NULL
GROUP BY EXTRACT(month FROM o.date)
ORDER BY EXTRACT(month FROM o.date);


## 3. Vistas

Crie uma vista que resuma as informações mais importantes sobre as vendas de produtos, combinando informações de diferentes tabelas do esquema de base de dados. A vista deve ter o seguinte esquema:

product_sales(sku, order_no, qty, total_price, year, month, day_of_month, day_of_week, city)

In [ ]:
%%sql
CREATE VIEW product_sales AS
SELECT
    p.SKU,
    o.order_no,
    c.qty,
    (c.qty * p.price) AS total_price,
    EXTRACT(YEAR FROM o.date) AS year,
    EXTRACT(MONTH FROM o.date) AS month,
    EXTRACT(DAY FROM o.date) AS day_of_month,
    EXTRACT(DOW FROM o.date) AS day_of_week,
    SPLIT_PART(cu.address, ',', 1) AS city
FROM
    product p
    JOIN contains c ON p.SKU = c.SKU
    JOIN orders o ON c.order_no = o.order_no
    JOIN customer cu ON o.cust_no = cu.cust_no;


## 4. Desenvolvimento de Aplicação

### Explicação da arquitetura da aplicação web, incluindo um link para uma versão de trabalho e as relações entre os vários ficheiros na pasta web/arquivos

Trata-se duma aplicação Flask com um banco de dados PostgreSQL como armazenamento persistente dos dados - a conexão com este é estabelecida através de psycopg (um adptador PostgreSQL para Python). A aplicação conta com Flask (uma framework web leve e flexível para Python), e utiliza o protocolo WSGI (Web Server Gateway Interface - fornecido pelo próprio Flask) para a comunicação entre a aplicação Flask e o servidor web.

Utilizamos uma pool de conexões para as gerenciar com o banco de dados de forma eficiente. A classe ConnectionPool do módulo psycopg_pool é utilizada para criar e gerenciar um pool de conexões PostgreSQL. Isso ajuda a reutilizar conexões e reduzir o overhead de criar uma nova conexão para cada request.

O módulo logging do Python configura o registo de logs, e é acompanhado pela função distConfig que configura as definições de logging, incluindo o formato do log, os handlers e os níveis de log. Os logs são transmitidos para o fluxo de erro padrão do Flask.

Utilizámos o @app.route (do Flask) para definir várias rotas - cada uma representa uma combinação específica de endpoint URL e método HTTP, e está associada a uma função Python que lida com o request HTTP correspondente, executando queries na base de dados e renderizando templates HTML. Estes templates são renderizados com mecanismos do Flask e com base no layout definido com Jinja no base.html. Com o Jinja, é-nos possível popular a página web com dados da base de dados, para listar produtos, encomendas, clientes e fornecedores.

A app inclui um mecanismo de tratamento de erros, que captura e lida com exceções que ocorram durante o processamento dos requests. Quando um erro é capturado, o template error.html é renderizado, mostrando a mensagem de erro.

Para o utilizador da app interagir com a base de dados, este depara-se com formulários (forms de html) cujos dados são processados e utilizados em queries enviadas para a base de dados, como INSERT, UPDATE, DELETE de registros.

A aplicação suporta várias APIs RESTful para realizar operações CRUD (Criar, Ler, Atualizar, Deletar) em entidades de clientes, fornecedores, produtos e encomendas. As APIs são definidas como funções de rota e correspondem a métodos HTTP específicos (GET, POST, etc.).

Resumindo, a aplicação segue a arquitetura típica de uma aplicação web Flask com um banco de dados PostgreSQL. Utiliza rotas para mapear os URLs e métodos HTTP para funções Python correspondentes, manipula requests e responses, renderiza templates HTML, faz consultas e operações na base de dados, e possui um mecanismo de tratamento de erros. Através das APIs RESTful, os users podem realizar operações de criação, leitura, atualização e exclusão de dados nas entidades de clientes, fornecedores, produtos e encomendas. Essa arquitetura permite a construção de uma aplicação web funcional e escalável.

## 5. Consultas OLAP

Usando a vista desenvolvida para a Questão 3, escreva duas consultas SQL que permitam analisar:

1. As quantidade e valores totais de venda de cada produto em 2022, globalmente, por cidade, por mês, dia do mês e dia da semana

In [ ]:
%%sql

/* global */

SELECT
    sku,
    SUM(qty) AS total_quantity,
    SUM(total_price) AS total_value
FROM
    product_sales
WHERE
    year = 2022
GROUP BY
    ROLLUP (sku);

/* por sku e city*/

SELECT
    sku,
    city,
    SUM(qty) AS total_quantity,
    SUM(total_price) AS total_value
FROM
    product_sales
WHERE
    year = 2022
GROUP BY
    ROLLUP (sku, city);

/* por sku e month */

SELECT
    sku,
    month,
    SUM(qty) AS total_quantity,
    SUM(total_price) AS total_value
FROM
    product_sales
WHERE
    year = 2022
GROUP BY
    ROLLUP (sku, month);

/* por sku e dia do mês */

SELECT
    sku,
    day_of_month,
    SUM(qty) AS total_quantity,
    SUM(total_price) AS total_value
FROM
    product_sales
WHERE
    year = 2022
GROUP BY
    ROLLUP (sku, day_of_month);

/* por sku e dia da semana */

SELECT
    sku,
    day_of_week,
    SUM(qty) AS total_quantity,
    SUM(total_price) AS total_value
FROM
    product_sales
WHERE
    year = 2022
GROUP BY
    ROLLUP (sku, day_of_week);

2. O valor médio diário das vendas de todos os produtos em 2022, globalmente, por mês e dia da semana

In [ ]:
%%sql

SELECT sku, month, day_of_week, AVG(total_price) AS mean_daily_sales
FROM product_sales
WHERE year = 2022
GROUP BY ROLLUP (sku, month, day_of_week);

## 6. Índices

Indique, com a devida justificação, que tipo de índice(s), sobre qual(is) atributo(s) e sobre qual(is) tabela(s) faria sentido criar, de forma a agilizar a execução de cada uma das seguintes consultas: 

### 6.1
SELECT order_no
FROM orders 
JOIN contains USING (order_no) 
JOIN product USING (SKU) 
WHERE price > 50 AND 
EXTRACT(YEAR FROM date) = 2023

### Tipo de Índice, Atributos & Justificação  

Para melhorar a performance da query, criaria os seguintes indíces:  

(1)Indíce na coluna price da tabela produto:  
**CREATE INDEX idx_product_price ON product (price);**  
Este indíce ajuda a optimizar a condição WHERE price > 50 condition, permitindo uma filtração eficiente.  

(2)Indíce na coluna date da tabela orders:  
**CREATE INDEX idx_orders_date ON orders (date);**  
Este indíce suporta a condição EXTRACT(YEAR FROM date) = 2023, permitindo uma procura rápida, e retorno das orders do ano especificado.  

(3) Indíce na coluna order_no da tabela contains:  
**CREATE INDEX idx_contains_order_no ON contains (order_no);**  
Este indíce facilita o join entre as tabelas orders e contains (através da coluna order_no).  

Os indíces escolhidos são indíces B-tree que se focam nas colunas usadas na filtragem (price e date) e no join (order_no). Ao indexar estas colunas, a base de dados consegue localizar as linhas relevantes, reduzindo a necessidade de percorrer totalmente as tabelas, e melhorando a performance da query.

Outros tipos de indíces, como indíces de chave composta, ou indíces de cobertura, poderiam não ser tão eficientes neste cenário, porque: indíces de chave composta são mais úteis quando múltiplas colunas são usadas em conjunto frequentemente, em condições de join ou de filtragem, mas nesta query, cada coluna tem um propósito distinto de filtragem/join; por sua vez, indíces de cobertura, que incluem todas as colunas relevantes à query, poderiam ser considerados, mas o custo ia sobrepôr-se aos potenciais benefícios deste indíce.

### 6.2
SELECT order_no, SUM(qty*price)
FROM contains 
JOIN product USING (SKU) 
WHERE name LIKE ‘A%’ 
GROUP BY order_no;

### Tipo de Índice, Atributos & Justificação

Para melhorar a performance da query, criaria os seguintes indíces:  

(1) Indíce na coluna name da tabela product:  
**CREATE INDEX idx_product_name ON product (name);**  
Este indíce ajuda a optimizar a operação WHERE name LIKE 'A%', ao returnar eficientemente as linhas com base na coluna name  

(2) Indíce na coluna SKU na tabela contains  
**CREATE INDEX idx_contains_SKU ON contains (SKU);**  
Este indíce ajuda a operação join entre as tabelas contains e product(através da coluna SKU)  

Estes dois indíces devem provenir as melhorias de performance mais significativas para esta query. Ao criar estes indíces B-tree, a base de dados consegue localizar as linhas relevantes baseado em name e SKU, reduzindo a necessidade de percorrer as tabelas na sua totalidade, e aumentando a velocidade de execução da query.  

Outros tipos de indíces, como indíces de hash, não são recomendados para este cenário. Isto porque este tipo de indíce é apropriado para procuras baseadas em igualdades, ao contrário do que acontece na query: uma operação de pattern matching (name LIKE 'A%'). Indíces B-tree são mais apropriados para optimizar pattern matching de strings, estando alinhavados com as necessidades da query.  